In [18]:
class Cell:
    def __init__(self, x: int, y: int) -> None:
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"Cell(x={self.x}, y={self.y})"

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, Cell) or __value is None:
            return False

        return self.x == __value.x and self.y == __value.y

    def __add__(self, __value: "Cell"):
        return Cell(self.x + __value.x, self.y + __value.y)

    def __sub__(self, __value: "Cell"):
        return Cell(self.x - __value.x, self.y - __value.y)

    @property
    def is_within_boundaries(self):
        return 0 <= self.x <= 7 and 0 <= self.y <= 5

In [19]:
from typing import NamedTuple, Union

POISONOUS_CELL = Union[Cell, None]


class MazeState(NamedTuple):
    slug_1: Cell
    slug_2: Cell
    poison_1: POISONOUS_CELL
    poison_2: POISONOUS_CELL
    poison_3: POISONOUS_CELL
    poison_4: POISONOUS_CELL

    def __str__(self) -> str:
        return f"MazeState(slug_1={self.slug_1}, slug_2={self.slug_2}, poison_1={self.poison_1}, poison_2={self.poison_2}, poison_3={self.poison_3}, poison_4={self.poison_4})"

    def __eq__(self, __value: "MazeState") -> bool:
        return self.slug_1 == __value.slug_1 and self.slug_2 == __value.slug_2

    @staticmethod
    def with_initial_state(s1: Cell, s2: Cell) -> "MazeState":
        return MazeState(s1, s2, None, None, None, None)

    def with_poisonous_cells(self, s1: Cell, s2: Cell, timestep: int) -> "MazeState":
        if timestep != 0 and timestep % 2 == 0:
            return MazeState(
                s1,
                s2,
                poison_1=self.poison_1,
                poison_2=None,
                poison_3=self.poison_3,
                poison_4=None,
            )

        return MazeState(
            s1,
            s2,
            poison_1=self.slug_1,
            poison_2=self.poison_1,
            poison_3=self.slug_2,
            poison_4=self.poison_3,
        )

In [20]:
class Node:
    def __init__(
        self, state: MazeState, parent: Union["Node", None], action: str
    ) -> None:
        self.state = state
        self.parent = parent
        self.action = action

In [21]:
from abc import ABC, abstractmethod


class Frontier(ABC):
    @abstractmethod
    def is_empty(self) -> bool:
        pass

    @abstractmethod
    def add_node(self, node: Node) -> None:
        pass

    @abstractmethod
    def remove_node(self) -> Node:
        pass

    @abstractmethod
    def has_state(self, state: MazeState) -> bool:
        pass


class StackFrontier(Frontier):
    def __init__(self) -> None:
        self.frontier: list[Node] = []

    @property
    def is_empty(self) -> bool:
        return len(self.frontier) == 0

    def add_node(self, node: Node) -> None:
        self.frontier.append(node)

    def remove_node(self) -> Node:
        if self.is_empty:
            raise Exception("Empty frontier")
        else:
            return self.frontier.pop()

    def has_state(self, state: MazeState) -> bool:
        return any(state == n.state for n in self.frontier)


class QueueFrontier(StackFrontier):
    def remove_node(self) -> Node:
        if self.is_empty:
            raise Exception("Empty frontier")
        else:
            return self.frontier.pop(0)

In [22]:
CANDIDATE_ACTION = tuple[str, MazeState]


def create_candidates(state: MazeState, timestep: int):
    slug1, slug2, poison1, _, poison3, _ = state

    candidates: list[CANDIDATE_ACTION] = [
        # They move in the same direction
        (
            "Both slugs moves up",
            state.with_poisonous_cells(
                slug1 + Cell(0, 1),
                slug2 + Cell(0, 1),
                timestep=timestep,
            ),
        ),
        (
            "Both slugs moves down",
            state.with_poisonous_cells(
                slug1 + Cell(0, -1),
                slug2 + Cell(0, -1),
                timestep=timestep,
            ),
        ),
        (
            "Both slugs moves left",
            state.with_poisonous_cells(
                slug1 + Cell(-1, 0),
                slug2 + Cell(-1, 0),
                timestep=timestep,
            ),
        ),
        (
            "Both slugs moves right",
            state.with_poisonous_cells(
                slug1 + Cell(1, 0),
                slug2 + Cell(1, 0),
                timestep=timestep,
            ),
        ),
        # They move in opposite directions
        # Slug 1 up
        (
            "Slug 1 moves up, slug 2 moves left",
            state.with_poisonous_cells(
                slug1 + Cell(0, 1),
                slug2 + Cell(-1, 0),
                timestep=timestep,
            ),
        ),
        (
            "Slug 1 moves up, slug 2 moves right",
            state.with_poisonous_cells(
                slug1 + Cell(0, 1),
                slug2 + Cell(1, 0),
                timestep=timestep,
            ),
        ),
        # Slug 1 down
        (
            "Slug 1 moves down, slug 2 moves left",
            state.with_poisonous_cells(
                slug1 + Cell(0, -1),
                slug2 + Cell(-1, 0),
                timestep=timestep,
            ),
        ),
        (
            "Slug 1 moves down, slug 2 moves right",
            state.with_poisonous_cells(
                slug1 + Cell(0, -1),
                slug2 + Cell(1, 0),
                timestep=timestep,
            ),
        ),
        # Slug 1 left
        (
            "Slug 1 moves left, slug 2 moves up",
            state.with_poisonous_cells(
                slug1 + Cell(-1, 0),
                slug2 + Cell(0, 1),
                timestep=timestep,
            ),
        ),
        (
            "Slug 1 moves left, slug 2 moves down",
            state.with_poisonous_cells(
                slug1 + Cell(-1, 0),
                slug2 + Cell(0, -1),
                timestep=timestep,
            ),
        ),
        # Slug 1 right
        (
            "Slug 1 moves right, slug 2 moves up",
            state.with_poisonous_cells(
                slug1 + Cell(1, 0),
                slug2 + Cell(0, 1),
                timestep=timestep,
            ),
        ),
        (
            "Slug 1 moves right, slug 2 moves down",
            state.with_poisonous_cells(
                slug1 + Cell(1, 0),
                slug2 + Cell(0, -1),
                timestep=timestep,
            ),
        ),
    ]

    return candidates

In [26]:
class SlugMaze:
    def __init__(
        self,
        initial_state: MazeState,
        goal_state: MazeState,
        wall: list[Cell],
    ) -> None:
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.wall = wall

        self.solution: Union[list[CANDIDATE_ACTION], None] = None
        self.timestep = 0

    def __str__(self) -> str:
        return f"SlugMaze(initial_state={self.initial_state}, goal_state={self.goal_state}, wall={self.wall}, solution={self.solution})"

    def print_solution(self) -> None:
        if self.solution is None:
            print("No solution")
        else:
            print(f"With {len(self.solution)} steps, we can reach the goal state:\n")
            for action, state in self.solution:
                s1, s2 = state.slug_1, state.slug_2
                print(f"{action} - Slug 1 at {s1} and Slug 2 at {s2}\n")

    def neighbors(self, state: MazeState, previous_action: str, goal: MazeState):
        # goal_slug1, goal_slug2, _, _, _, _ = goal

        candidates: list[CANDIDATE_ACTION] = create_candidates(state, self.timestep)
        self.timestep += 1
        result: list[CANDIDATE_ACTION] = []

        for action, candidate in candidates:
            slug1, slug2, poison1, poison2, poison3, poison4 = candidate
            poisonous_cells = [poison1, poison2, poison3, poison4]

            if (
                slug1 not in self.wall
                and slug2 not in self.wall
                and slug1 not in poisonous_cells
                and slug2 not in poisonous_cells
            ):  # not heading into bad cells
                if (
                    slug1.is_within_boundaries
                    and slug2.is_within_boundaries
                    and action != previous_action
                ):  # ensure we're still on the board
                    result.append((action, candidate))

        return result

    def solve(self, frontier: StackFrontier) -> None:
        self.num_explored = 0

        start = Node(state=self.initial_state, parent=None, action="None")
        frontier.add_node(start)

        self.explored: list[MazeState] = []

        while True:
            if frontier.is_empty:
                raise Exception("No solution")

            node = frontier.remove_node()
            self.num_explored += 1

            print(f"{self.num_explored} - {node.state}\n")

            if node.state == self.goal_state:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = list(zip(actions, cells))
                return

            self.explored.append(node.state)

            for action, state in self.neighbors(
                node.state, node.action, self.goal_state
            ):
                if state not in self.explored and not frontier.has_state(state):
                    child = Node(state=state, parent=node, action=action)
                    frontier.add_node(child)

In [24]:
wall: list[Cell] = [Cell(3, 0), Cell(3, 1), Cell(3, 2), Cell(4, 2), Cell(2, 2)]

goal_state = MazeState(
    slug_1=Cell(4, 0),
    slug_2=Cell(6, 0),
    poison_1=None,
    poison_2=None,
    poison_3=None,
    poison_4=None,
)

with_initial_state = MazeState.with_initial_state(s1=Cell(0, 0), s2=Cell(0, 4))
maze = SlugMaze(initial_state=with_initial_state, goal_state=goal_state, wall=wall)
stack_frontier = StackFrontier()
maze.solve(stack_frontier)

1 - MazeState(slug_1=Cell(x=0, y=0), slug_2=Cell(x=0, y=4), poison_1=None, poison_2=None, poison_3=None, poison_4=None)

2 - MazeState(slug_1=Cell(x=1, y=0), slug_2=Cell(x=0, y=3), poison_1=Cell(x=0, y=0), poison_2=None, poison_3=Cell(x=0, y=4), poison_4=None)

3 - MazeState(slug_1=Cell(x=1, y=1), slug_2=Cell(x=1, y=3), poison_1=Cell(x=1, y=0), poison_2=Cell(x=0, y=0), poison_3=Cell(x=0, y=3), poison_4=Cell(x=0, y=4))

4 - MazeState(slug_1=Cell(x=2, y=1), slug_2=Cell(x=1, y=2), poison_1=Cell(x=1, y=0), poison_2=None, poison_3=Cell(x=0, y=3), poison_4=None)

5 - MazeState(slug_1=Cell(x=1, y=1), slug_2=Cell(x=1, y=1), poison_1=Cell(x=2, y=1), poison_2=Cell(x=1, y=0), poison_3=Cell(x=1, y=2), poison_4=Cell(x=0, y=3))

6 - MazeState(slug_1=Cell(x=1, y=0), slug_2=Cell(x=0, y=1), poison_1=Cell(x=2, y=1), poison_2=None, poison_3=Cell(x=1, y=2), poison_4=None)

7 - MazeState(slug_1=Cell(x=2, y=0), slug_2=Cell(x=0, y=0), poison_1=Cell(x=1, y=0), poison_2=Cell(x=2, y=1), poison_3=Cell(x=0, y=1),

In [25]:
maze.print_solution()

With 74 steps, we can reach the goal state:

Slug 1 moves right, slug 2 moves down - Slug 1 at Cell(x=1, y=0) and Slug 2 at Cell(x=0, y=3)

Slug 1 moves up, slug 2 moves right - Slug 1 at Cell(x=1, y=1) and Slug 2 at Cell(x=1, y=3)

Slug 1 moves right, slug 2 moves down - Slug 1 at Cell(x=2, y=1) and Slug 2 at Cell(x=1, y=2)

Slug 1 moves left, slug 2 moves down - Slug 1 at Cell(x=1, y=1) and Slug 2 at Cell(x=1, y=1)

Both slugs moves left - Slug 1 at Cell(x=0, y=1) and Slug 2 at Cell(x=0, y=1)

Slug 1 moves right, slug 2 moves down - Slug 1 at Cell(x=1, y=1) and Slug 2 at Cell(x=0, y=0)

Slug 1 moves up, slug 2 moves right - Slug 1 at Cell(x=1, y=2) and Slug 2 at Cell(x=1, y=0)

Slug 1 moves up, slug 2 moves left - Slug 1 at Cell(x=1, y=3) and Slug 2 at Cell(x=0, y=0)

Slug 1 moves right, slug 2 moves up - Slug 1 at Cell(x=2, y=3) and Slug 2 at Cell(x=0, y=1)

Slug 1 moves up, slug 2 moves right - Slug 1 at Cell(x=2, y=4) and Slug 2 at Cell(x=1, y=1)

Slug 1 moves right, slug 2 moves 